In [1]:
 ! pip install -q kaggle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/kaggle"
# /content/gdrive/My Drive/Kaggle is the path where kaggle.json is present in the Google Drive

In [4]:
%cd '/content/gdrive/MyDrive/kaggle'

/content/gdrive/MyDrive/kaggle


In [5]:
!kaggle datasets download -d ameyamote030/einterface-image-dataset

einterface-image-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# Importing needed libraries
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils

# Preprocessing

## specifying file paths for train, test and val data

In [7]:
path_train='/content/gdrive/My Drive/kaggle/eINTERFACE_2021_Image/train'
path_test= '/content/gdrive/My Drive/kaggle/eINTERFACE_2021_Image/test'
path_val= '/content/gdrive/My Drive/kaggle/eINTERFACE_2021_Image/val'


In [14]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]
x_val=[]
y_val=[]

In [15]:
# dictionary to encode categories(emotions) into numbers
emotion_encode={'Anger':0,'Disgust':1,'Fear':2,'Happiness':3,'Sadness':4,'Surprise':5}

In [13]:
for i in sorted(os.listdir(path_train)):
  print(i)

Anger
Disgust
Fear
Happiness
Sadness
Surprise


## Preparing train data

In [16]:
dims = 128 # dimensions of final images to get uniformity

for i in sorted(os.listdir(path_train)):


    path=os.path.join(path_train,i)
    for j in os.listdir(path):


      img=cv2.imread(os.path.join(path,j),0) 
      img=cv2.resize(img,(dims,dims)) 
      x_train.append(img)
      y_train.append(emotion_encode[i]) 
        
print('Number of Training images:',len(x_train))
print('Number of Labels for training images:',len(y_train))

Number of Training images: 11475
Number of Labels for training images: 11475


## preparing test data

In [17]:
dims = 128 # dimensions of final images to get uniformity

for i in sorted(os.listdir(path_test)):
    path=os.path.join(path_test,i)
    for j in os.listdir(path):
        img=cv2.imread(os.path.join(path,j),0) 
        img=cv2.resize(img,(dims,dims)) 
        x_test.append(img)
        y_test.append(emotion_encode[i])
        
print('Number of Training images:',len(x_test))
print('Number of Labels for training images:',len(y_test))

Number of Training images: 1438
Number of Labels for training images: 1438


## preparing val data

In [18]:
dims = 128 # dimensions of final images to get uniformity

for i in sorted(os.listdir(path_val)):
    path=os.path.join(path_val,i)
    for j in os.listdir(path):
        img=cv2.imread(os.path.join(path,j),0) 
        img=cv2.resize(img,(dims,dims)) 
        x_val.append(img)
        y_val.append(emotion_encode[i]) 
        
print('Number of Training images:',len(x_val))
print('Number of Labels for training images:',len(y_val))

Number of Training images: 1433
Number of Labels for training images: 1433


## Normalising train, test and val data

In [19]:
x_train= np.array(x_train, dtype="float") / 255.0
x_test=np.array(x_test,dtype="float") / 255.0
x_val= np.array(x_val, dtype="float") / 255.0

y_train= np.array(y_train) 
y_test=np.array(y_test)
y_val=np.array(y_val)

y_train=np_utils.to_categorical(y_train)
y_test=np_utils.to_categorical(y_test)
y_val=np_utils.to_categorical(y_val)


## getting 2D matrices of each image

In [20]:
x_train_reshaped=x_train.reshape(x_train.shape[0],dims,dims,1)
x_test_reshaped=x_test.reshape(x_test.shape[0],dims,dims,1)
x_val_reshaped=x_val.reshape(x_val.shape[0],dims,dims,1)
print(x_train_reshaped.shape)
print(x_test_reshaped.shape)
print(x_val_reshaped.shape)

(11475, 128, 128, 1)
(1438, 128, 128, 1)
(1433, 128, 128, 1)


In [21]:
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(11475, 6)
(1438, 6)
(1433, 6)


In [22]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

# First simple network

## Defining first network

In [33]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), input_shape=(dims,dims,1), activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(16, (3,3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(6,activation="softmax"))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 16)      160       
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 32)        9

## Compiling first *network*

In [35]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.1)
# Early stopping callback
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

## Training first network

In [36]:
model_history=model.fit(x_train_reshaped,y_train,batch_size=256,epochs=50,validation_data=(x_val_reshaped,y_val),shuffle=True, callbacks=[callback])

Epoch 1/50
45/45 [==============================] - 468s 10s/step - loss: 770886.6221 - accuracy: 0.1695 - val_loss: 1.7948 - val_accuracy: 0.1647
Epoch 2/50
45/45 [==============================] - 449s 10s/step - loss: 1.7939 - accuracy: 0.1653 - val_loss: 1.7920 - val_accuracy: 0.1675
Epoch 3/50
45/45 [==============================] - 447s 10s/step - loss: 1.7929 - accuracy: 0.1673 - val_loss: 1.7924 - val_accuracy: 0.1675
Epoch 4/50
45/45 [==============================] - 448s 10s/step - loss: 1.7936 - accuracy: 0.1664 - val_loss: 1.7927 - val_accuracy: 0.1654
Epoch 5/50
45/45 [==============================] - 447s 10s/step - loss: 1.7940 - accuracy: 0.1653 - val_loss: 1.7931 - val_accuracy: 0.1675


## Checking performace of first network on test data

In [37]:
# Testing the model
model.evaluate(x_test_reshaped,y_test)

45/45 [==============================] - 13s 292ms/step - loss: 1.7931 - accuracy: 0.1676


[1.793056845664978, 0.16759388148784637]

# Second network

## Defining second network(little more complex than firs network)

In [39]:
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3,3), input_shape=(dims,dims,1), activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(64, (3,3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(128, (3,3), activation='relu',padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1024, activation="relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(512,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(6,activation="softmax"))

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 128, 32)      320       
_________________________________________________________________
dropout_3 (Dropout)          (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 64, 64, 64)       

## Compiling second network

In [43]:
from keras.optimizers import SGD
epoch=50
learning_rate=0.01
decay_rate=learning_rate/epoch
momentum=0.8
sgd=SGD(learning_rate=learning_rate,momentum=momentum,decay=decay_rate)

In [44]:
# callback for EarlyStopping if val_loss does not improve over 3 epochs 
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [45]:
 model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

## Training second network

In [46]:

model_history=model.fit(x_train_reshaped,y_train,batch_size=32,epochs=epoch,validation_data=(x_val_reshaped,y_val),shuffle=True, callbacks=[callback])

Epoch 1/50
359/359 [==============================] - 1276s 4s/step - loss: 1.6991 - accuracy: 0.2603 - val_loss: 1.3960 - val_accuracy: 0.4703
Epoch 2/50
359/359 [==============================] - 1265s 4s/step - loss: 1.2060 - accuracy: 0.5207 - val_loss: 0.9913 - val_accuracy: 0.6434
Epoch 3/50
359/359 [==============================] - 1271s 4s/step - loss: 0.8759 - accuracy: 0.6577 - val_loss: 0.6831 - val_accuracy: 0.7558
Epoch 4/50
359/359 [==============================] - 1268s 4s/step - loss: 0.6559 - accuracy: 0.7514 - val_loss: 0.6009 - val_accuracy: 0.7906
Epoch 5/50
359/359 [==============================] - 1268s 4s/step - loss: 0.4877 - accuracy: 0.8241 - val_loss: 0.5159 - val_accuracy: 0.8053
Epoch 6/50
359/359 [==============================] - 1270s 4s/step - loss: 0.3955 - accuracy: 0.8536 - val_loss: 0.3256 - val_accuracy: 0.8828
Epoch 7/50
359/359 [==============================] - 1265s 4s/step - loss: 0.3108 - accuracy: 0.8851 - val_loss: 0.3071 - val_accuracy:

## Checking performance of second network on test data

In [47]:
# Testing the model
model.evaluate(x_test_reshaped,y_test)

45/45 [==============================] - 34s 761ms/step - loss: 0.1497 - accuracy: 0.9541


[0.1497226059436798, 0.9541029334068298]

# Second network with more conv2D layers and optimizer SGD with momentum and adaptive learning rate gives good accuracy.  